In [ ]:
import cv2
from ultralytics import RTDETR, YOLO
from ultralytics.solutions import object_counter
import numpy as np

# Инициализация модели детекции
# model = RTDETR("rtdetr-l.pt")  # RT-DETR large
model = YOLO("../app/models/weights/yolov8n.pt")  # или YOLOv8nano для более быстрой работы

# Открытие видеофайла или видеопотока
video_path = "../datasets/videos/adpiskunov.mp4"  # или 0 для веб-камеры
cap = cv2.VideoCapture(video_path)

# Получение параметров видео
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Создание VideoWriter для сохранения результата
output_path = "tracked_people.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Цикл обработки кадров
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    print(model(frame))
    # Детекция и трекинг объектов (только люди - класс 0 в COCO)
    results = model.track(
        frame,
        persist=True,
        tracker="bytetrack.yaml",  # используем встроенный трекер
        classes=[0],  # только класс 'person'
        verbose=False,
        device='gpu'
    )
    
    # Визуализация результатов
    annotated_frame = results[0].plot()
    
    # Отображение ID треков
    if results[0].boxes.id is not None:
        track_ids = results[0].boxes.id.int().cpu().tolist()
        boxes = results[0].boxes.xyxy.cpu().numpy()
        
        for box, track_id in zip(boxes, track_ids):
            x1, y1, x2, y2 = map(int, box)
            cv2.putText(
                annotated_frame,
                f"ID: {track_id}",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 255, 0),
                2
            )
    
    # Запись кадра в выходной файл
    out.write(annotated_frame)
    
    # Отображение результата
    cv2.imshow("People Tracking", annotated_frame)
    
    # Выход по нажатию 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Трекинг завершен. Результат сохранен в {output_path}")


0: 384x640 1 person, 1 chair, 1 laptop, 41.3ms
Speed: 1.6ms preprocess, 41.3ms inference, 71.8ms postprocess per image at shape (1, 3, 384, 640)
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwi

KeyboardInterrupt: 

: 